# All user in training, but only 80% of purchase history

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import ParameterGrid

# 1. Load the data - reusing your existing code
x_train = pd.read_csv('X_train.csv', delimiter=',')
y_train = pd.read_csv('y_train.csv', delimiter=',')
basket_features = pd.read_csv('../basket_features.csv')

# Initialize encoders dictionary
encoders = {}

# Merge the data
merged_data = pd.merge(x_train, y_train, on='user_id')

# Encode features directly in the merged DataFrame
for col in ['location', 'gender', 'education', 'invest_goal', 'age_group']:
    encoder = LabelEncoder()
    encoder.fit(merged_data[col])
    merged_data[f'{col}_encoded'] = encoder.transform(merged_data[col])
    encoders[col] = encoder

# Encode basket names
all_basket_names = basket_features['basket_name'].unique()
basket_encoder = LabelEncoder()
basket_encoder.fit(all_basket_names)
num_baskets = len(basket_encoder.classes_)
merged_data['basket_encoded'] = basket_encoder.transform(merged_data['basket_name'])

# Convert to numpy arrays
X = merged_data[[f'{col}_encoded' for col in encoders.keys()]].values
y = merged_data['basket_encoded'].values

# Dataset class - reusing your existing code
class RecommendationDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), self.y[idx]

# Updated MLP model to support variable hidden layers
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        
        # Create a list of layers
        layers = []
        
        # Input layer to first hidden layer
        layers.append(nn.Linear(input_size, hidden_sizes[0]))
        layers.append(nn.ReLU())
        
        # Create additional hidden layers
        for i in range(1, len(hidden_sizes)):
            layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            layers.append(nn.ReLU())
        
        # Output layer
        layers.append(nn.Linear(hidden_sizes[-1], output_size))
        
        # Create a sequential container
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

# Updated function to train and validate model with specific parameters
def train_and_validate(X, y, hidden_sizes, learning_rate, batch_size, num_epochs):
    # Create dataset
    dataset = RecommendationDataset(X, y)
    
    # Split into train/validation
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize model
    input_size = X.shape[1]
    output_size = num_baskets
    model = MLP(input_size, hidden_sizes, output_size)
    
    # Use GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
        
        # Validation
        model.eval()
        total_val_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                
                total_val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += batch_y.size(0)
                correct += (predicted == batch_y).sum().item()
        
        # Calculate metrics
        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = total_val_loss / len(val_loader)
        accuracy = correct / total
        
        # Print progress every few epochs
        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, "
                  f"Val Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}")
    
    return model, avg_val_loss, accuracy

# Updated Grid Search to support neural network architectures
def grid_search(param_grid):
    # Create all parameter combinations
    grid = list(ParameterGrid(param_grid))
    print(f"Total combinations to try: {len(grid)}")
    
    # Store results
    results = []
    best_val_loss = float('inf')
    best_params = None
    
    # Try each combination
    for i, params in enumerate(grid):
        print(f"\nCombination {i+1}/{len(grid)}:")
        print(f"Parameters: {params}")
        print(f"Architecture: {len(params['hidden_sizes'])} hidden layers with sizes {params['hidden_sizes']}")
        
        # Train and validate with these parameters
        model, val_loss, accuracy = train_and_validate(
            X, y, 
            hidden_sizes=params['hidden_sizes'],
            learning_rate=params['learning_rate'],
            batch_size=params['batch_size'],
            num_epochs=params['num_epochs']
        )
        
        # Store results
        results.append({
            'params': params,
            'val_loss': val_loss,
            'accuracy': accuracy
        })
        
        # Check if this is the best model so far
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_params = params
            print(f"New best model found! Validation Loss: {best_val_loss:.4f}, Accuracy: {accuracy:.4f}")
    
    # Print final best results
    print("\n" + "="*50)
    print("Grid Search Complete!")
    print(f"Best Validation Loss: {best_val_loss:.4f}")
    print("Best Parameters:")
    for key, value in best_params.items():
        print(f"  {key}: {value}")
    
    return best_params, results

# Updated train_final_model function to support variable hidden layers
def train_final_model(best_params):
    # Parameters
    input_size = X.shape[1]
    hidden_sizes = best_params['hidden_sizes']
    output_size = num_baskets
    batch_size = best_params['batch_size']
    num_epochs = best_params['num_epochs']
    learning_rate = best_params['learning_rate']
    
    # Create dataset and dataloader
    dataset = RecommendationDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Initialize model with variable hidden layers
    model = MLP(input_size, hidden_sizes, output_size)
    
    # Use GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    print("\nTraining final model with best parameters:")
    print(f"Architecture: {len(hidden_sizes)} hidden layers with sizes {hidden_sizes}")
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        for batch_x, batch_y in dataloader:
            # Move data to device
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            # Forward pass
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Print progress
        if (epoch + 1) % 2 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")
    
    return model, basket_encoder

# 1. Perform grid search
print("Starting Grid Search...")
# Define parameter grid with different network architectures
param_grid = {
    'hidden_sizes': [
        [32],
        [32, 32],
        [32, 64, 32],
    ],
    'learning_rate': [0.01, 0.005, 0.002],
    'batch_size': [32],
    'num_epochs': [30]
}
best_params, all_results = grid_search(param_grid)

# 2. Train the final model with the best parameters
final_model, basket_encoder = train_final_model(best_params)

# 3. Save the model and encoders
torch.save({
    'model_state_dict': final_model.state_dict(),
    'best_params': best_params,
    'encoders': encoders,
    'basket_encoder': basket_encoder
}, 'best_recommendation_model.pth')

print("\nModel training complete and saved to 'best_recommendation_model.pth'")

Starting Grid Search...
Total combinations to try: 9

Combination 1/9:
Parameters: {'batch_size': 32, 'hidden_sizes': [32], 'learning_rate': 0.01, 'num_epochs': 30}
Architecture: 1 hidden layers with sizes [32]
Epoch [5/30], Train Loss: 3.5096, Val Loss: 3.5388, Accuracy: 0.1229
Epoch [10/30], Train Loss: 3.4063, Val Loss: 3.4425, Accuracy: 0.1211
Epoch [15/30], Train Loss: 3.3627, Val Loss: 3.4209, Accuracy: 0.1101
Epoch [20/30], Train Loss: 3.3388, Val Loss: 3.4049, Accuracy: 0.1216
Epoch [25/30], Train Loss: 3.3190, Val Loss: 3.4542, Accuracy: 0.1154
Epoch [30/30], Train Loss: 3.3151, Val Loss: 3.3889, Accuracy: 0.1250
New best model found! Validation Loss: 3.3889, Accuracy: 0.1250

Combination 2/9:
Parameters: {'batch_size': 32, 'hidden_sizes': [32], 'learning_rate': 0.005, 'num_epochs': 30}
Architecture: 1 hidden layers with sizes [32]
Epoch [5/30], Train Loss: 3.4964, Val Loss: 3.5738, Accuracy: 0.1242
Epoch [10/30], Train Loss: 3.3819, Val Loss: 3.4924, Accuracy: 0.1133
Epoch [1

In [64]:
import collections
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load test data
x_test = pd.read_csv('X_test.csv', delimiter=',')  # Contains user features
y_test = pd.read_csv('y_test.csv', delimiter=',')  # Contains user_id and basket_name

# Create merged train data for finding baskets users have already bought
train_data = pd.merge(x_train, y_train, on='user_id')

# Get a list of all unique baskets
unique_baskets = list(basket_encoder.classes_)

# Function to predict basket probabilities for a user
def predict_user_baskets(user_id, model, exclude_baskets=None):
    # Find the user in the original data
    user_data = x_train[x_train['user_id'] == user_id]
    if len(user_data) == 0:
        return []
    
    # Encode features on-the-fly for this user
    feature_vector = []
    for col in encoders.keys():
        encoded_value = encoders[col].transform([user_data[col].iloc[0]])[0]
        feature_vector.append(encoded_value)
    
    # Convert to tensor and predict
    x = torch.tensor(feature_vector, dtype=torch.float32).unsqueeze(0).to(device)
    
    # Get predictions
    with torch.no_grad():
        logits = final_model(x)
        probabilities = torch.softmax(logits, dim=1).squeeze(0)
    
    # Convert to numpy for easier manipulation
    probs = probabilities.cpu().numpy()
    
    # Create a list of (basket_idx, probability) tuples
    basket_probs = [(i, probs[i]) for i in range(len(probs))]
    
    # If exclude_baskets is provided, filter them out
    if exclude_baskets is not None:
        basket_probs = [(idx, prob) for idx, prob in basket_probs if idx not in exclude_baskets]
    
    # Sort by probability (descending)
    basket_probs.sort(key=lambda x: x[1], reverse=True)
    
    return basket_probs

# Evaluation function - logic remains the same
def evaluate_model(model, x_test, y_test, train_data, top_k_values=[1, 2, 3]):
    # Dictionary to store precision, recall, and F1 values
    precision_at_k = collections.defaultdict(list)
    recall_at_k = collections.defaultdict(list)
    f1_at_k = collections.defaultdict(list)
    
    # Get all unique test users
    test_user_ids = x_test['user_id'].unique()
    
    # For each user in the test set
    for user_id in test_user_ids:
        # Find baskets this user has invested in from test data (ground truth)
        user_test_data = y_test[y_test['user_id'] == user_id]
        
        # Skip if user has no test data
        if len(user_test_data) == 0:
            continue
            
        user_positive_test_baskets = set(basket_encoder.transform(user_test_data['basket_name']))
        
        # If no positive test baskets, skip this user
        if len(user_positive_test_baskets) == 0:
            continue
        
        # Find baskets the user has already invested in from train data
        user_train_data = train_data[train_data['user_id'] == user_id]
        user_invested_train_baskets = set()
        
        if len(user_train_data) > 0:
            user_invested_train_baskets = set(basket_encoder.transform(user_train_data['basket_name']))
        
        # Get predictions for this user (excluding already purchased baskets)
        basket_probs = predict_user_baskets(user_id, model, exclude_baskets=user_invested_train_baskets)
        
        # Skip if no predictions
        if len(basket_probs) == 0:
            continue
            
        # Calculate precision and recall at different k values
        for k in top_k_values:
            # Ensure k doesn't exceed number of predictions
            effective_k = min(k, len(basket_probs))
            
            # Skip if no predictions
            if effective_k == 0:
                continue
            
            # Get top-k recommended baskets
            top_k_recs = [idx for idx, _ in basket_probs[:effective_k]]
            
            # Calculate relevant items among top-k recommendations
            true_positives = len(set(top_k_recs) & user_positive_test_baskets)
            
            # Precision = relevant recommended / all recommended
            precision = true_positives / effective_k
            
            # Recall = relevant recommended / all relevant
            recall = true_positives / len(user_positive_test_baskets)
            
            # F1 score = 2 * (precision * recall) / (precision + recall)
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            precision_at_k[k].append(precision)
            recall_at_k[k].append(recall)
            f1_at_k[k].append(f1)
    
    # Calculate average precision, recall, and F1 for each k
    results = {}
    print("\nEvaluation Metrics for Top-K Recommendations:")
    for k in top_k_values:
        avg_precision = np.mean(precision_at_k[k]) if precision_at_k[k] else 0
        avg_recall = np.mean(recall_at_k[k]) if recall_at_k[k] else 0
        avg_f1 = np.mean(f1_at_k[k]) if f1_at_k[k] else 0
        
        print(f"\nMetrics for k={k}:")
        print(f"Precision@{k}: {avg_precision:.4f}")
        print(f"Recall@{k}: {avg_recall:.4f}")
        print(f"F1@{k}: {avg_f1:.4f}")
        

        print(f"Number of users evaluated: {len(precision_at_k[k])}")
        
        results[k] = {
            'precision': avg_precision,
            'recall': avg_recall,
            'f1': avg_f1,
            'num_users': len(precision_at_k[k])
        }
    
    return results

# Run the evaluation
evaluation_results = evaluate_model(model, x_test, y_test, train_data)


Evaluation Metrics for Top-K Recommendations:

Metrics for k=1:
Precision@1: 0.2257
Recall@1: 0.1675
F1@1: 0.1869
Number of users evaluated: 988

Metrics for k=2:
Precision@2: 0.1842
Recall@2: 0.2647
F1@2: 0.2110
Number of users evaluated: 988

Metrics for k=3:
Precision@3: 0.1488
Recall@3: 0.3214
F1@3: 0.1982
Number of users evaluated: 988


# 80% user in training with full purchase history

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import ParameterGrid, GroupShuffleSplit

# 1. Load the data
x_train = pd.read_csv('X_train.csv', delimiter=',')
y_train = pd.read_csv('y_train.csv', delimiter=',')
x_test = pd.read_csv('X_test.csv', delimiter=',')
y_test = pd.read_csv('y_test.csv', delimiter=',')
basket_features = pd.read_csv('../basket_features.csv')

# 2. Merge training and test data
x_combined = pd.concat([x_train, x_test], ignore_index=True)
y_combined = pd.concat([y_train, y_test], ignore_index=True)

# 3. Merge features and targets
merged_data = pd.merge(x_combined, y_combined, on='user_id')

# 4. Initialize encoders dictionary
encoders = {}

# 5. Encode categorical features
for col in ['location', 'gender', 'education', 'invest_goal', 'age_group']:
    encoder = LabelEncoder()
    encoder.fit(merged_data[col])
    merged_data[f'{col}_encoded'] = encoder.transform(merged_data[col])
    encoders[col] = encoder

# 6. Encode basket names
all_basket_names = basket_features['basket_name'].unique()
basket_encoder = LabelEncoder()
basket_encoder.fit(all_basket_names)
num_baskets = len(basket_encoder.classes_)
merged_data['basket_encoded'] = basket_encoder.transform(merged_data['basket_name'])

# 7. Split data by user_id: 80% of users in training, 20% in test
splitter = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_indices, test_indices = next(splitter.split(merged_data, groups=merged_data['user_id']))

# Create new training and test sets
new_x_train = merged_data.iloc[train_indices]
new_x_test = merged_data.iloc[test_indices]

# Verify the split worked correctly
train_users = set(new_x_train['user_id'])
test_users = set(new_x_test['user_id'])
print(f"Total unique users: {len(set(merged_data['user_id']))}")
print(f"Users in training set: {len(train_users)}")
print(f"Users in test set: {len(test_users)}")
print(f"Overlap between train and test users: {len(train_users.intersection(test_users))}")
print(f"Training set size: {len(new_x_train)}")
print(f"Test set size: {len(new_x_test)}")

# 8. Prepare features and targets for model training
X_train = new_x_train[[f'{col}_encoded' for col in encoders.keys()]].values
y_train = new_x_train['basket_encoded'].values
X_test = new_x_test[[f'{col}_encoded' for col in encoders.keys()]].values
y_test = new_x_test['basket_encoded'].values

# Dataset class
class RecommendationDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), self.y[idx]

# MLP model with variable hidden layers
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        
        # Create a list of layers
        layers = []
        
        # Input layer to first hidden layer
        layers.append(nn.Linear(input_size, hidden_sizes[0]))
        layers.append(nn.ReLU())
        
        # Create additional hidden layers
        for i in range(1, len(hidden_sizes)):
            layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            layers.append(nn.ReLU())
        
        # Output layer
        layers.append(nn.Linear(hidden_sizes[-1], output_size))
        
        # Create a sequential container
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

# Function to train and evaluate model
def train_and_evaluate(X_train, y_train, X_test, y_test, hidden_sizes, learning_rate, batch_size, num_epochs):
    # Create datasets
    train_dataset = RecommendationDataset(X_train, y_train)
    test_dataset = RecommendationDataset(X_test, y_test)
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize model
    input_size = X_train.shape[1]
    output_size = num_baskets
    model = MLP(input_size, hidden_sizes, output_size)
    
    # Use GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
        
        # Test evaluation
        model.eval()
        total_test_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch_x, batch_y in test_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                
                total_test_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += batch_y.size(0)
                correct += (predicted == batch_y).sum().item()
        
        # Calculate metrics
        avg_train_loss = total_train_loss / len(train_loader)
        avg_test_loss = total_test_loss / len(test_loader)
        accuracy = correct / total
        
        # Print progress every few epochs
        if (epoch + 1) % 2 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, "
                  f"Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.4f}")
    
    return model, avg_test_loss, accuracy

# Grid Search for hyperparameter optimization
def grid_search(param_grid):
    # Create all parameter combinations
    grid = list(ParameterGrid(param_grid))
    print(f"Total combinations to try: {len(grid)}")
    
    # Store results
    results = []
    best_test_loss = float('inf')
    best_params = None
    
    # Try each combination
    for i, params in enumerate(grid):
        print(f"\nCombination {i+1}/{len(grid)}:")
        print(f"Parameters: {params}")
        print(f"Architecture: {len(params['hidden_sizes'])} hidden layers with sizes {params['hidden_sizes']}")
        
        # Train and evaluate with these parameters
        model, test_loss, accuracy = train_and_evaluate(
            X_train, y_train, X_test, y_test,
            hidden_sizes=params['hidden_sizes'],
            learning_rate=params['learning_rate'],
            batch_size=params['batch_size'],
            num_epochs=params['num_epochs']
        )
        
        # Store results
        results.append({
            'params': params,
            'test_loss': test_loss,
            'accuracy': accuracy
        })
        
        # Check if this is the best model so far
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            best_params = params
            print(f"New best model found! Test Loss: {best_test_loss:.4f}, Accuracy: {accuracy:.4f}")
    
    # Print final best results
    print("\n" + "="*50)
    print("Grid Search Complete!")
    print(f"Best Test Loss: {best_test_loss:.4f}")
    print("Best Parameters:")
    for key, value in best_params.items():
        print(f"  {key}: {value}")
    
    return best_params, results

# Function to train final model with best parameters
def train_final_model(best_params):
    # Parameters
    input_size = X_train.shape[1]
    hidden_sizes = best_params['hidden_sizes']
    output_size = num_baskets
    batch_size = best_params['batch_size']
    num_epochs = best_params['num_epochs']
    learning_rate = best_params['learning_rate']
    
    # Create combined dataset for final training
    X_all = np.vstack((X_train, X_test))
    y_all = np.concatenate((y_train, y_test))
    
    dataset = RecommendationDataset(X_all, y_all)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Initialize model with best parameters
    model = MLP(input_size, hidden_sizes, output_size)
    
    # Use GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    print("\nTraining final model with best parameters:")
    print(f"Architecture: {len(hidden_sizes)} hidden layers with sizes {hidden_sizes}")
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        for batch_x, batch_y in dataloader:
            # Move data to device
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            # Forward pass
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Print progress
        if (epoch + 1) % 2 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")
    
    return model, basket_encoder

# Define parameter grid with different network architectures
param_grid = {
    'hidden_sizes': [
        [32],
        [32, 32],
        [32, 64, 32],
    ],
    'learning_rate': [0.001, 0.0005, 0.0002],
    'batch_size': [32],
    'num_epochs': [10]
}

# Perform grid search
print("Starting Grid Search...")
best_params, all_results = grid_search(param_grid)

# Train the final model with the best parameters
final_model, basket_encoder = train_final_model(best_params)

Total unique users: 994
Users in training set: 795
Users in test set: 199
Overlap between train and test users: 0
Training set size: 27616
Test set size: 6605
Starting Grid Search...
Total combinations to try: 9

Combination 1/9:
Parameters: {'batch_size': 32, 'hidden_sizes': [32], 'learning_rate': 0.001, 'num_epochs': 10}
Architecture: 1 hidden layers with sizes [32]
Epoch [2/10], Train Loss: 3.8987, Test Loss: 4.0850, Accuracy: 0.0902
Epoch [4/10], Train Loss: 3.7046, Test Loss: 4.0740, Accuracy: 0.0969
Epoch [6/10], Train Loss: 3.6469, Test Loss: 4.1286, Accuracy: 0.0948
Epoch [8/10], Train Loss: 3.6095, Test Loss: 4.1975, Accuracy: 0.0955
Epoch [10/10], Train Loss: 3.5825, Test Loss: 4.2566, Accuracy: 0.0975
New best model found! Test Loss: 4.2566, Accuracy: 0.0975

Combination 2/9:
Parameters: {'batch_size': 32, 'hidden_sizes': [32], 'learning_rate': 0.0005, 'num_epochs': 10}
Architecture: 1 hidden layers with sizes [32]
Epoch [2/10], Train Loss: 4.0557, Test Loss: 4.2035, Accurac

In [62]:
import collections
import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to predict basket probabilities for a user
def predict_user_baskets(user_features, model, basket_encoder, encoders, exclude_baskets=None):
    # Encode features for this user
    feature_vector = []
    for col in encoders.keys():
        col_name = f'{col}_encoded'
        if col_name in user_features:
            # If already encoded
            feature_vector.append(user_features[col_name])
        else:
            # If needs encoding on-the-fly
            encoded_value = encoders[col].transform([user_features[col]])[0]
            feature_vector.append(encoded_value)
    
    # Convert to tensor and predict
    x = torch.tensor(feature_vector, dtype=torch.float32).unsqueeze(0).to(device)
    
    # Get predictions
    with torch.no_grad():
        logits = model(x)
        probabilities = torch.softmax(logits, dim=1).squeeze(0)
    
    # Convert to numpy for easier manipulation
    probs = probabilities.cpu().numpy()
    
    # Create a list of (basket_idx, probability) tuples
    basket_probs = [(i, probs[i]) for i in range(len(probs))]
    
    # If exclude_baskets is provided, filter them out
    if exclude_baskets is not None:
        basket_probs = [(idx, prob) for idx, prob in basket_probs if idx not in exclude_baskets]
    
    # Sort by probability (descending)
    basket_probs.sort(key=lambda x: x[1], reverse=True)
    
    return basket_probs

# Evaluation function for the new data split
def evaluate_model(model, new_x_test, new_x_train, basket_encoder, encoders, top_k_values=[1, 2, 3, 5, 10]):
    # Dictionary to store precision, recall, and F1 values
    precision_at_k = collections.defaultdict(list)
    recall_at_k = collections.defaultdict(list)
    f1_at_k = collections.defaultdict(list)
    
    # Get all unique test users
    test_user_ids = new_x_test['user_id'].unique()
    
    # For each user in the test set
    for user_id in test_user_ids:
        # Get all test data for this user
        user_test_data = new_x_test[new_x_test['user_id'] == user_id]
        
        # Skip if user has no test data
        if len(user_test_data) == 0:
            continue
            
        # Get the baskets this user has invested in (ground truth)
        user_positive_test_baskets = set(user_test_data['basket_encoded'])
        
        # If no positive test baskets, skip this user
        if len(user_positive_test_baskets) == 0:
            continue
        
        # Find baskets the user has already invested in from train data (if any)
        user_train_data = new_x_train[new_x_train['user_id'] == user_id]
        user_invested_train_baskets = set()
        
        if len(user_train_data) > 0:
            # This should be empty since we split by user, but included for completeness
            user_invested_train_baskets = set(user_train_data['basket_encoded'])
        
        # Get the first row of user data (features are the same for all rows of the same user)
        user_features = user_test_data.iloc[0]
        
        # Get predictions for this user (excluding already purchased baskets if any)
        basket_probs = predict_user_baskets(
            user_features, 
            model, 
            basket_encoder, 
            encoders, 
            exclude_baskets=user_invested_train_baskets
        )
        
        # Skip if no predictions
        if len(basket_probs) == 0:
            continue
            
        # Calculate precision and recall at different k values
        for k in top_k_values:
            # Ensure k doesn't exceed number of predictions
            effective_k = min(k, len(basket_probs))
            
            # Skip if no predictions
            if effective_k == 0:
                continue
            
            # Get top-k recommended baskets
            top_k_recs = [idx for idx, _ in basket_probs[:effective_k]]
            
            # Calculate relevant items among top-k recommendations
            true_positives = len(set(top_k_recs) & user_positive_test_baskets)
            
            # Precision = relevant recommended / all recommended
            precision = true_positives / effective_k
            
            # Recall = relevant recommended / all relevant
            recall = true_positives / len(user_positive_test_baskets)
            
            # F1 score = 2 * (precision * recall) / (precision + recall)
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            precision_at_k[k].append(precision)
            recall_at_k[k].append(recall)
            f1_at_k[k].append(f1)
    
    # Calculate average precision, recall, and F1 for each k
    results = {}
    print("\nEvaluation Metrics for Top-K Recommendations:")
    for k in top_k_values:
        if not precision_at_k[k]:
            print(f"\nNo data for k={k}")
            continue
            
        avg_precision = np.mean(precision_at_k[k])
        avg_recall = np.mean(recall_at_k[k])
        avg_f1 = np.mean(f1_at_k[k])
        
        print(f"\nMetrics for k={k}:")
        print(f"Precision@{k}: {avg_precision:.4f}")
        print(f"Recall@{k}: {avg_recall:.4f}")
        print(f"F1@{k}: {avg_f1:.4f}")
        
        # Also print number of users evaluated
        print(f"Number of users evaluated: {len(precision_at_k[k])}")
        
        results[k] = {
            'precision': avg_precision,
            'recall': avg_recall,
            'f1': avg_f1,
            'num_users': len(precision_at_k[k])
        }
    
    return results

# After training the model, you would call the evaluation like this:
def run_evaluation(model, new_x_test, new_x_train, basket_encoder, encoders):
    print("Running evaluation on the test set...")
    evaluation_results = evaluate_model(
        model=model,
        new_x_test=new_x_test,
        new_x_train=new_x_train,
        basket_encoder=basket_encoder,
        encoders=encoders,
        top_k_values=[1, 2, 3]
    )
    return evaluation_results


# After training your final model
evaluation_results = run_evaluation(
    model=final_model,
    new_x_test=new_x_test,
    new_x_train=new_x_train,
    basket_encoder=basket_encoder,
    encoders=encoders
)

Running evaluation on the test set...

Evaluation Metrics for Top-K Recommendations:

Metrics for k=1:
Precision@1: 0.6080
Recall@1: 0.1149
F1@1: 0.1909
Number of users evaluated: 199

Metrics for k=2:
Precision@2: 0.5829
Recall@2: 0.2229
F1@2: 0.3157
Number of users evaluated: 199

Metrics for k=3:
Precision@3: 0.5159
Recall@3: 0.2911
F1@3: 0.3644
Number of users evaluated: 199
